In [15]:
!which python

/home/mritter/anaconda3/envs/tf_gpu_test04/bin/python


In [16]:
import keras

In [17]:
# IF IT DOES NOT WORK, MAY NEED TO RESTART COMPUTER

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

# confirm PyTorch sees the GPU
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0
print(cuda.get_device_name(cuda.current_device()))

AssertionError: 

# Load Data

In [ ]:
# Download data
# https://files.pushshift.io/hackernews/ (first file)
# https://files.pushshift.io/hackernews/HNI_2006-10.bz2
# Manifest: https://files.pushshift.io/hackernews/HNI_total_items_by_month.txt

In [4]:
from requests import get  # to make GET request


def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = get(url)
        # write to file
        file.write(response.content)

download('https://files.pushshift.io/hackernews/HNI_total_items_by_month.txt', 'data/manifest.txt')

In [5]:
! head data/manifest.txt

        61 HNI_2006-10
         1 HNI_2006-12
      1549 HNI_2007-02
      6305 HNI_2007-03
     10335 HNI_2007-04
      7516 HNI_2007-05
      6036 HNI_2007-06
      6410 HNI_2007-07
     10841 HNI_2007-08
     12371 HNI_2007-09


In [6]:
import pandas as pd
with open('data/manifest.txt', 'r') as f:
    manifest = pd.Series(f.read().split('\n')).map(str.split)
    manifest = manifest[manifest.map(len) > 0].reset_index(drop=True)
manifest.tail()

134    [209738, HNI_2018-02]
135    [237342, HNI_2018-03]
136    [237609, HNI_2018-04]
137    [237646, HNI_2018-05]
138        [17172781, total]
dtype: object

In [4]:
# import os.path

# url_format = 'https://files.pushshift.io/hackernews/{}.bz2'
# file_format = 'data/{}.bz2'
# for size, filename in manifest:
#     if os.path.isfile(file_format.format(filename)): continue 
#     download(url_format.format(filename), file_format.format(filename))
    

In [8]:
! ls data/*2011-1*bz2

data/HNI_2011-10.bz2  data/HNI_2011-11.bz2  data/HNI_2011-12.bz2


In [9]:
from string import ascii_letters
ascii_letters.index('a')

0

# Preprocess

In [20]:
import dask.bag as db
from dask.distributed import Client, progress
import json, re
client = Client(n_workers=8, threads_per_worker=2, memory_limit='6GB')
client

def comment_filter(record):
    return (record['type'] == 'comment' 
            and record.get('deleted', None) == None
            and record.get('text', None) != None)

def text_transformation(record):
    text = record['text'].lower()
    text = re.sub('http.*\w',' <LINK> ',text)
    un = record['by'].lower()
    return (un, text)

b = db.read_text('data/*bz2').map(json.loads).filter(comment_filter)\
      .map(text_transformation)

# temp = b.to_dataframe()
# temp.index = ['un', 'text']
# temp2 = temp.tail()
# temp2

In [12]:
from IPython.display import HTML, display

for row in b.random_sample(.00001):
    display(HTML(row[1]))
    print('--')

--


--


--


--


--


...

--


--


--


--


--


In [30]:
b.count().compute()

10763434

In [44]:
b.filter(lambda x: x[0] == 'patio11').count().compute()

9472

# Format for use

In [21]:
%%time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

MAX_SEQUENCE_LENGTH = 1000
SAMPLE_LENGTH = 100000
sl = int(SAMPLE_LENGTH/2)
non_target_texts = b.map(lambda x: x[1]).take(sl, npartitions=-1)
target_texts = b.filter(lambda x: x[0] == 'patio11').map(lambda x: x[1]).take(sl, npartitions=-1)
texts = non_target_texts + target_texts

tokenizer = Tokenizer(num_words=MAX_SEQUENCE_LENGTH)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray([0]*len(non_target_texts) + [1]*len(target_texts)))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 57663 unique tokens.
Shape of data tensor: (59472, 1000)
Shape of label tensor: (100000, 2)
CPU times: user 50 s, sys: 8.25 s, total: 58.3 s
Wall time: 3min 21s


In [33]:
# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.2

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


In [34]:
x_train[:5]

array([[  0,   0,   0, ...,   0,   0, 236],
       [  0,   0,   0, ..., 516,  14, 380],
       [  0,   0,   0, ...,   3, 310, 750],
       [  0,   0,   0, ...,   4,  85, 531],
       [  0,   0,   0, ...,  67,  11, 136]], dtype=int32)

In [35]:
y_train[:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [29]:
# def tokenize(record):
#     text = record[1]
#     textl = text.split()
#     textl += ['']*100
#     textl = textl[:100]
#     return (record[0], *textl)

# test_sample = b.take(10, compute=False)
# test_sample = test_sample.map(tokenize)
# test_sample.to_dataframe().compute()

In [128]:
# import glob, time
# from tqdm import tqdm
# import bz2, json
# from json import JSONDecodeError

# bigstring = b''
# for filename in tqdm(glob.glob('data/*bz2')):
#     bigstring += bz2.BZ2File(filename).read()
    
# bigstring[-500:]

In [129]:
# Convert to HDF5
# import bz2, json
# from json import JSONDecodeError
# b = bz2.BZ2File('data/HNI_2006-10.bz2')
# bs = b.read()
# bs[:100]

In [130]:
# bj = []
# for l in bs.split(b'\n'):
#     try:
#         bj.append(json.loads(l))
#     except JSONDecodeError:
#         pass
# bjc = [x for x in bj if x['type'] == 'comment']
# bjc[:2]

In [24]:
# Tag
# Maybe things like links should be tagged
# import re
# for i in range(len(bjc)):
#     bjc[i]['text'] = bjc[i]['text'].lower()
#     bjc[i]['text'] = re.sub('http.*\w',' <LINK> ',bjc[i]['text'])
#     bjc[i]['text'] = re.sub('\n|\r|"|\'|\?|&#34;|\.|\,','',bjc[i]['text'])

# [x['text'] for x in bjc]

In [11]:
# # Tokenize
# text_list = []
# user_list = []
# for (text, user) in ((x['text'], x['by']) for x in bjc):
#     text_list.append(text.split())
#     user_list.append(user)


In [12]:
# # W2V
# from gensim.test.utils import common_texts, get_tmpfile
# from gensim.models import Word2Vec
# EMBEDDING_DIM = 100

# w2vmodel = Word2Vec(text_list, size=EMBEDDING_DIM, window=5, min_count=1, workers=4)
# w2vmodel.save("word2vec.w2vmodel")

In [30]:
# # Index
# SEQ_LEN = 10

# tok_list = []
# for text in text_list:
#     tok_list.append([])
#     for tok in text:
#         try:
#             tok_list[-1].append(1+w2vmodel.wv.index2word.index(tok))
#         except:
#             tok_list[-1].append(0)
            
#     tok_list[-1] = tok_list[-1] + [0]*SEQ_LEN 
#     tok_list[-1] = tok_list[-1][:SEQ_LEN] 
            
            
# tok_list[5]

In [31]:
# import numpy as np
# x_train = np.array(tok_list)
# x_train

In [32]:
# from keras.utils import to_categorical

# y_train = to_categorical([int(x == 'pg') for x in user_list])
# y_train

In [43]:
%%time
# first, build index mapping words in the embeddings set
# to their embedding vector
import os 
BASE_DIR = '/home/mritter/code/twitter_nlp/newsgroups_data/'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove')

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.
CPU times: user 11.2 s, sys: 1.09 s, total: 12.3 s
Wall time: 10.8 s


In [39]:
len(tokenizer.word_index)

57663

In [44]:
# prepare embedding matrix

num_distinct_words = len(tokenizer.word_index) + 1  # For <UNKNOWN> 
EMBEDDING_DIM = 100  # Dimensions to represent each token

embedding_matrix = np.zeros((num_distinct_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_distinct_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [46]:
embedding_matrix.shape #[:5]

(57664, 100)

In [48]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

embedding_layer = Embedding(num_distinct_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [50]:
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


In [51]:
model.fit(x_train, y_train,
          batch_size=32, #128,
          epochs=2,
          validation_data=(x_val, y_val))


Train on 47578 samples, validate on 11894 samples
Epoch 1/2
 1568/47578 [..............................] - ETA: 8:00 - loss: 0.4995 - acc: 0.8355

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 5391 was killed by unknown signal
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING -

KeyboardInterrupt: 

In [17]:
# # Build model
# from keras.models import Sequential
# from keras.layers import Embedding, Flatten, Dense

# # embedding_layer = Embedding(len(model.wv.index2word) + 1,
# #                             EMBEDDING_DIM,
# #                             weights=[embedding_matrix],
# #                             input_length=MAX_SEQUENCE_LENGTH,
# #                             trainable=False)

# # ValueError: The shape of the input to "Flatten" is not fully defined (got (None, 100). 
# # Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.


# model = Sequential()
# # embedding_layer = w2vmodel.wv.get_keras_embedding()
# # model.add(embedding_layer(EMBEDDING_DIM, input_dim=SEQ_LEN))
# model.add(Embedding(1000, 64, input_length=10))
# # the model will take as input an integer matrix of size (batch, input_length).
# # the largest integer (i.e. word index) in the input should be
# # no larger than 999 (vocabulary size).
# # now model.output_shape == (None, 10, 64), where None is the batch dimension.

# model.add(Flatten())#input_dim=EMBEDDING_DIM*SEQ_LEN))
# model.add(Dense(units=5, activation='relu'))
# model.add(Dense(units=2, activation='softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer='sgd',
#               metrics=['accuracy'])

In [18]:
# Train Model
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
12/12 [==============================] - 3s 224ms/step - loss: 0.6918 - acc: 0.5833
Epoch 2/5
12/12 [==============================] - 0s 773us/step - loss: 0.6913 - acc: 0.5833
Epoch 3/5
12/12 [==============================] - 0s 725us/step - loss: 0.6909 - acc: 0.5833
Epoch 4/5
12/12 [==============================] - 0s 570us/step - loss: 0.6904 - acc: 0.5833
Epoch 5/5
12/12 [==============================] - 0s 467us/step - loss: 0.6901 - acc: 0.5833


In [19]:
# Test Model
predictions = model.predict(x_train)
print(predictions)
loss_and_metrics = model.evaluate(x_train, y_train, batch_size=1)
print(y_train.mean(axis=0))
print(loss_and_metrics)

[[0.5030554  0.49694455]
 [0.5023337  0.49766627]
 [0.50491714 0.4950828 ]
 [0.51636297 0.483637  ]
 [0.49919084 0.50080913]
 [0.49966925 0.50033075]
 [0.5031254  0.4968746 ]
 [0.49960157 0.5003984 ]
 [0.5059884  0.4940116 ]
 [0.50231534 0.49768466]
 [0.50238585 0.49761418]
 [0.49969432 0.5003057 ]]
12/12 [==============================] - 0s 3ms/step
[0.5833333  0.41666666]
[0.6897125691175461, 0.5833333333333334]


In [10]:
# New Data